## The Goal: Start testing out how to compute HCF 
<b>Author:</b> Meg D. Fowler<br>
<b>Date:</b> 29 Sept 2020 <br><br>

In [23]:
# Import libraries 
import comet as cm 
import numpy as np 
import xarray as xr 
import pickle
import pandas as pd
import datetime 
import datetime 
import time 

# Plotting utils 
import matplotlib.pyplot as plt 
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.util


## Read in some data?

<b>What data do we need?</b> <br>
Vertical profiles of: temperature (T), specific humidity (Q), geopotential height (zg in CESM2, Z3 in other runs), and pressure (P). <br>
In addition, need lowest level temperature, specfic humidity, height, and pressure - so basically T2m, Q2m, PSfc, and 2m height. <br><br>
<b>Units:</b><br>
Temperature --> K <br>
Height      --> m <br>
Sp. Humidity -> kg/kg <br>
Pressure    --> Pa  <br><br>


Start with a focus on 1980 to test things out




In [3]:
dataDir = '/Users/mdfowler/Documents/Analysis/Coupling_initial/data/hrSim_CONUS/'

Tpr_file = dataDir+'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.cam.h1.1980_hrT-UTCsel.nc'
Zpr_file = dataDir+'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.cam.h1.1980_hrZ3-UTCsel.nc'
Qpr_file = dataDir+'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.cam.h1.1980_hrQ-UTCsel.nc'
Ppr_file = dataDir+'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.cam.h1.1979-1981_hrP-levels-UTCsel.nc'

# ----------- Open files -------------
print('Read in profile of...')
Tpr = xr.open_dataset(Tpr_file, decode_times=True)
Tpr['time'] = Tpr.indexes['time'].to_datetimeindex()
print('.....T')
Zpr = xr.open_dataset(Zpr_file, decode_times=True)
Zpr['time'] = Zpr.indexes['time'].to_datetimeindex()
print('.....Z3')
Qpr = xr.open_dataset(Qpr_file, decode_times=True)
Qpr['time'] = Qpr.indexes['time'].to_datetimeindex()
print('.....Q')
Ppr = xr.open_dataset(Ppr_file, decode_times=True)
Ppr['time'] = Ppr.indexes['time'].to_datetimeindex()
print('.....P')

# ----------- Isolate 1980 in Ppr -----
dates1980     = pd.DatetimeIndex(Tpr['time'].values)
datesPpr      = pd.DatetimeIndex(Ppr['time'].values)
iTimes        = np.where( (datesPpr>=(dates1980[0])) & (datesPpr<=dates1980[-1]) )[0]

Ppr_sel       = Ppr.isel(time=iTimes)


Read in profile of...


<ipython-input-3-712b54a28809>:11: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  Tpr['time'] = Tpr.indexes['time'].to_datetimeindex()
<ipython-input-3-712b54a28809>:14: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  Zpr['time'] = Zpr.indexes['time'].to_datetimeindex()
<ipython-input-3-712b54a28809>:17: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  Qpr['t

.....T
.....Z3
.....Q
.....P


<ipython-input-3-712b54a28809>:20: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  Ppr['time'] = Ppr.indexes['time'].to_datetimeindex()


### To make things a bit easier, assemble everything into a single dataframe, ordered with the surface level first 

In [4]:
ds_Full = Tpr 
ds_Full['Qpr'] = (('time','lev','lat','lon'), Qpr.Q)
ds_Full['Zpr'] = (('time','lev','lat','lon'), Zpr.Z3)
ds_Full['Ppr'] = (('time','lev','lat','lon'), Ppr_sel.PRESSURE)


In [5]:
# Pick out 12 UTC only 
ds_utc12 = ds_Full.where( ds_Full.UTC_hr==12.0 , drop=True )


**Pick out test location/time**

In [33]:
# Convert to dataframe for one point in space and time: 
iLat  = 10
iLon  = 30 
iTime = 10 

DF = ds_utc12.isel(lat=iLat,lon=iLon,time=iTime).to_dataframe()

# Flip order of levels so that surface comes first (required for function)
DF = DF.reindex(index=DF.index[::-1])



In [34]:
# Define length of dimensions 
nLev  = len(ds_utc12.lev)
print(nLev)


32


## Wondering about using f2py...

In [35]:
# Define variable names 
Tname = 'T'
Qname = 'Qpr'
Zname = 'Zpr'
Pname = 'Ppr'

# Profile starting at level above sfc
tmp_in   = DF[Tname].values[1::]
qhum_in  = DF[Qname].values[1::]
hgt_in   = DF[Zname].values[1::]
press_in = DF[Pname].values[1::]

# Sfc values set as first level values 
t2m      = DF[Tname].values[0]
q2m      = DF[Qname].values[0]
h2m      = DF[Zname].values[0]
psfc     = DF[Pname].values[0]

# Number of levels to worry about in actual "sounding"
nlev1 = nLev-1 

In [36]:
#--------------------------------
# Compute CTP-HiLow
#--------------------------------
from comet.metrics.fortran import hcf

TBM, BCLH, BCLP, TDEF = hcf.hcf_vars_calc.hcfcalc(nlev1,
                                                    tmp_in, 
                                                    press_in,
                                                    qhum_in, 
                                                    hgt_in, 
                                                    t2m, 
                                                    psfc,
                                                    q2m, 
                                                    h2m)


# TBM  : *** buoyant mixing pot. temp (convective threshold) [K]
# BCLH : *** height above ground of convective threshold [m]
# BCLP : *** pressure of convective threshold level [Pa]
# TDEF : *** potential temperature deficit need to initiate [K] 

# !! Note that evaluation metrics are not returned with this call. 


In [37]:
TBM

317.9294128417969

## Compute using above approach (CoMeT f2py version)

In [38]:
# Define dimensions 
nLat  = len(ds_utc12.lat)
nLon  = len(ds_utc12.lon)
nTime = len(ds_utc12.time)

In [39]:
# Define variable names 
Tname = 'T'
Qname = 'Qpr'
Zname = 'Zpr'
Pname = 'Ppr'

# Number of levels to worry about in actual "sounding"
nLev  = len(ds_utc12.lev)
nlev1 = nLev-1 

In [41]:
# Define empty arrays to save things into 
TBM_all   = np.full([nTime,nLat,nLon], np.nan)
BCLH_all  = np.full([nTime,nLat,nLon], np.nan)
BCLP_all  = np.full([nTime,nLat,nLon], np.nan)
TDEF_all  = np.full([nTime,nLat,nLon], np.nan)

# Time how long this takes... 
t_start     = time.time()


for iLat in range(nLat):
    for iLon in range(nLon):
        for iT in range(nTime):
            
            # Pick out specific point and time period 
            DF = ds_utc12.isel(lat=iLat,lon=iLon,time=iT).to_dataframe()
            
            # Flip order of levels so that surface comes first (required for function)
            DF = DF.reindex(index=DF.index[::-1])
            
            # Compute HCF variables

            TBM_all[iT,iLat,iLon], BCLH_all[iT,iLat,iLon], BCLP_all[iT,iLat,iLon], TDEF_all[iT,iLat,iLon] = hcf.hcf_vars_calc.hcfcalc(nlev1, 
                                                                                                    DF[Tname].values[1::], 
                                                                                                    DF[Pname].values[1::],
                                                                                                    DF[Qname].values[1::], 
                                                                                                    DF[Zname].values[1::], 
                                                                                                    DF[Tname].values[0], 
                                                                                                    DF[Pname].values[0],
                                                                                                    DF[Qname].values[0], 
                                                                                                    DF[Zname].values[0])
    print('Done with lat %i of %i ' % (iLat, nLat))
        

print('Time elapsed for all points and times: %.3f sec' % (time.time() - t_start))



Done with lat 0 of 43 
Done with lat 1 of 43 
Done with lat 2 of 43 
Done with lat 3 of 43 
Done with lat 4 of 43 
Done with lat 5 of 43 
Done with lat 6 of 43 
Done with lat 7 of 43 
Done with lat 8 of 43 
Done with lat 9 of 43 
Done with lat 10 of 43 
Done with lat 11 of 43 
Done with lat 12 of 43 
Done with lat 13 of 43 
Done with lat 14 of 43 
Done with lat 15 of 43 
Done with lat 16 of 43 
Done with lat 17 of 43 
Done with lat 18 of 43 
Done with lat 19 of 43 
Done with lat 20 of 43 
Done with lat 21 of 43 
Done with lat 22 of 43 
Done with lat 23 of 43 
Done with lat 24 of 43 
Done with lat 25 of 43 
Done with lat 26 of 43 
Done with lat 27 of 43 
Done with lat 28 of 43 
Done with lat 29 of 43 
Done with lat 30 of 43 
Done with lat 31 of 43 
Done with lat 32 of 43 
Done with lat 33 of 43 
Done with lat 34 of 43 
Done with lat 35 of 43 
Done with lat 36 of 43 
Done with lat 37 of 43 
Done with lat 38 of 43 
Done with lat 39 of 43 
Done with lat 40 of 43 
Done with lat 41 of 43 
Do

In [43]:
#  Save the array with TBM_all 
saveDir  = '/Users/mdfowler/Documents/Analysis/Coupling_initial/Coupling_CAM6CLM5/processed_data/'
saveFile = 'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.HCF-TBM_1980_CONUS_12utc.p' 

pickle.dump( TBM_all, open( saveDir+saveFile, "wb" ), protocol=4 )


In [44]:
#  Save the array with TBM_all 
saveDir  = '/Users/mdfowler/Documents/Analysis/Coupling_initial/Coupling_CAM6CLM5/processed_data/'
saveFile = 'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.HCF-BCLH_1980_CONUS_12utc.p' 

pickle.dump( BCLH_all, open( saveDir+saveFile, "wb" ), protocol=4 )


In [45]:
#  Save the array with TBM_all 
saveDir  = '/Users/mdfowler/Documents/Analysis/Coupling_initial/Coupling_CAM6CLM5/processed_data/'
saveFile = 'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.HCF-BCLP_1980_CONUS_12utc.p' 

pickle.dump( BCLP_all, open( saveDir+saveFile, "wb" ), protocol=4 )



In [46]:
#  Save the array with TBM_all 
saveDir  = '/Users/mdfowler/Documents/Analysis/Coupling_initial/Coupling_CAM6CLM5/processed_data/'
saveFile = 'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.HCF-TDEF_1980_CONUS_12utc.p' 

pickle.dump( TDEF_all, open( saveDir+saveFile, "wb" ), protocol=4 )


## Start attempt to compute from scratch 

In [45]:
# Define variable names 
Tname = 'T'
Qname = 'Qpr'
Zname = 'Zpr'
Pname = 'Ppr'

# Profile starting at level above sfc
tmp_in   = DF[Tname].values[1::]
qhum_in  = DF[Qname].values[1::]
hgt_in   = DF[Zname].values[1::]
press_in = DF[Pname].values[1::]

# Sfc values set as first level values 
t2m      = DF[Tname].values[0]
q2m      = DF[Qname].values[0]
h2m      = DF[Zname].values[0]
psfc     = DF[Pname].values[0]

# Number of levels to worry about in actual "sounding"
nlev1 = nLev-1 


In [46]:
# -----------------------------------------------
#    Set constants 
# -----------------------------------------------
p_ref  = 1e5 
Lv     = 2.5e6 
cp     = 1005.7
R_cp   = 287.04/1005.7

grav   = 9.81 
Rd     = 287.04
pi     = np.pi
cp_g   = cp/grav
Lv_g   = Lv/grav
r2d    = 180./pi

by100  = 1e2
t0     = 273.15, 
ep     = 0.622
es0    = 6.11
a      = 17.269
b      = 35.86
onemep = 1.0 - ep


In [51]:
# -----------------------------------------------
#    Initiate empty arrays 
# -----------------------------------------------
shdef = np.full([nlev1+1], np.nan)
lhdef = np.full([nlev1+1], np.nan)
eadv  = np.full([nlev1+1], np.nan)
rhoh  = np.full([nlev1+1], np.nan)
pbar  = np.full([nlev1+1], np.nan)
qdef  = np.full([nlev1+1], np.nan)
qmix  = np.full([nlev1+1], np.nan)

qsat   = np.full([nlev1+1], np.nan)
dpress = np.full([nlev1+1], np.nan) 
qbar   = np.full([nlev1+1], np.nan)
logp   = np.full([nlev1+1], np.nan)
hbar   = np.full([nlev1+1], np.nan)
tbar   = np.full([nlev1+1], np.nan)
tmp_k  = np.full([nlev1+1], np.nan)
press  = np.full([nlev1+1], np.nan)
pot_k  = np.full([nlev1+1], np.nan)
hgt    = np.full([nlev1+1], np.nan)
qhum   = np.full([nlev1+1], np.nan)

pot_diff = np.full([nlev1+1], np.nan)
eadv_0   = np.full([nlev1+1], np.nan)
xaxis    = np.full([nlev1+1], np.nan)
xaxis1   = np.full([nlev1+1], np.nan)
yaxis    = np.full([nlev1+1], np.nan)
yaxis1   = np.full([nlev1+1], np.nan)
integral = np.full([nlev1+1], np.nan)
below    = np.full([nlev1+1], np.nan)




In [58]:
# -----------------------------------------------
#    Store temp working arrays and initialize 
# -----------------------------------------------
nlev      = nlev1+1 

tmp_k[1:] = tmp_in 
hgt[1:]   = hgt_in
qhum[1:]  = qhum_in
press[1:] = press_in 

tmp_k[0] = t2m 
hgt[0]   = h2m
qhum[0]  = q2m 
press[0] = psfc


In [59]:
# ----- Run a few checks on pressure (but most of this is unfortunately on the user ) ------- 

# Check that pressure levels aren't greater than the surface pressure 
iProblem = np.where(press[1:] >= psfc)[0]

if len(iProblem)>0:
    print('***** ERROR: lowest pressure level > surface pressure *****')

# Check ordering of Plev: 
if press[0]<press[-1]:
    print('***** ERROR: pressure levels should be reversed *****') 

# Check units of pressure 
if psfc<=2000.0: 
    print('**** ERROR: pressures should be in Pa, not hPa *****') 


In much of code below, I have the calculation in .f90 from CoMeT ncl script commented out. 

In [91]:
# -----------------------------------------------
#    Compute column potential temperature 
# -----------------------------------------------
pot_k = tmp_k * (p_ref/press)**(R_cp)

# -----------------------------------------------
#    Ignore missing data levels when calculating midpoints 
#     (shouldn't be an issue for model data)
# -----------------------------------------------
hbar = hgt
pbar = press
tbar = tmp_k

# -----------------------------------------------
#    Compute middle layer specific humidity average [kg/kg]
#    1st layer = the 2m sp. humidity above, then layer averages above 
# -----------------------------------------------
qbar = qhum 
qbar[1:nlev] = ( (qhum[1:nlev]*np.log(press[1:nlev]) + qhum[0:nlev-1]*np.log(press[0:nlev-1]) )  / 
                  np.log(press[1:nlev]*press[0:nlev-1]))
# qbar(2:nlev)    = ((qhum(2:nlev  )*log(press(2:nlev  ))  + &
#                                   qhum(1:nlev-1)*log(press(1:nlev-1))) / &
#                                   log(press(2:nlev)* press(1:nlev-1)))


# -----------------------------------------------
#    Compute pressure difference of each layer 
# -----------------------------------------------
if dpress[0]<=0: 
    dpress[0] = 1.     # Set to 1 Pa because h2m is likely zero 
else:
    dpress[0]   = (psfc / (Rd * t2m * ((1. + (q2m/ep)) / (1. + q2m)) )) * grav * h2m 
    #dpress(1)  =  (psfc / (Rd * t2m * ((1. + (q2m/ep)) / (1. + q2m)) )) * grav * h2m
    
# Model data shouldn't have any missing, so not using this line:
#    where( pbar(1:nlev-1).ne.missing  .and.  pbar(2:nlev).ne.missing )
dpress[1:nlev] = press[0:nlev-1] - press[1:nlev]

# -----------------------------------------------
#    Compute log pressure ot linearize it for slope calculation 
# -----------------------------------------------
logp = np.log(pbar)

# -----------------------------------------------
#    Compute mixed layer sp. humidity and column density [kg/kg]
# -----------------------------------------------
qmix = qbar * dpress/grav
rhoh = dpress/grav

for izz in range(nlev-1):
    zz = izz+1    # .f90 is: do zz = 2, nlev; so going to increase index by one 
    if (np.isfinite(qmix[zz]) & np.isfinite(qmix[zz-1])): 
        qmix[zz] = qmix[zz-1] + qmix[zz]
        
    if (np.isfinite(rhoh[zz]) & np.isfinite(rhoh[zz-1]) ):
        rhoh[zz] = rhoh[zz-1] + rhoh[zz]
